In [1]:
from fastapi import FastAPI
from starlette.exceptions import HTTPException
from starlette.middleware.cors import CORSMiddleware
from starlette.status import HTTP_422_UNPROCESSABLE_ENTITY

#from .api.api_v1.api import router as api_router
from core.config import ALLOWED_HOSTS, PROJECT_NAME, API_PORT
from core.errors import http_422_error_handler, http_error_handler
from db.mongodb_connect import close_mongo_connection, connect_to_mongo
from db.mongodb import AsyncIOMotorClient, get_database
import asyncio

In [2]:
app = FastAPI(title=PROJECT_NAME)

if not ALLOWED_HOSTS:
    ALLOWED_HOSTS = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=ALLOWED_HOSTS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


app.add_event_handler("startup", connect_to_mongo)
app.add_event_handler("shutdown", close_mongo_connection)

app.add_exception_handler(HTTPException, http_error_handler)
app.add_exception_handler(HTTP_422_UNPROCESSABLE_ENTITY, http_422_error_handler)

In [3]:

#from api.api import router as api_router
#app.include_router(api_router)


In [4]:
from fastapi.staticfiles import StaticFiles
app.mount("/static", StaticFiles(directory="static"), name="static")

In [5]:
from fastapi.responses import HTMLResponse
@app.get("/")
def home():
    with open("./static/index.html") as f:
        html = "".join(f.readlines())
    return HTMLResponse(content=html, status_code= 200)

In [6]:
@app.get("/index2")
def fun():
    with open("./static/index2.html") as f:
        html = "".join(f.readlines())
    return HTMLResponse(content=html, status_code= 200)

In [7]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=API_PORT)

INFO:     Started server process [8719]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:13525 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://0.0.0.0:13525 (Press CTRL+C to quit)
INFO:     140.122.104.65:0 - "GET /robots.txt HTTP/1.1" 200 OK
INFO:     140.122.104.65:0 - "GET / HTTP/1.1" 200 OK
INFO:     140.122.104.65:0 - "GET /static/main.css HTTP/1.1" 200 OK
INFO:     140.122.104.65:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     140.122.104.65:0 - "GET / HTTP/1.1" 200 OK
INFO:     140.122.104.65:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [8719]
INFO:uvicorn.error:Fin

In [ ]:
@app.post("/request_phone_code", tags=["user"])
def request_phone_code():
    pass

@app.post("/help_add_bot", tags=["user"])
def request_phone_code():
    pass

In [ ]:
@app.get("/")
def index_page():
    pass

In [ ]:
await connect_to_mongo()
from db.mongodb import get_database

In [ ]:
db = await get_database()

In [ ]:
col = db["test"]["test"]

In [ ]:
result = await col.insert_one({"test": "TTT"})

In [ ]:
result.inserted_id